# Performance analysis

In this notebook all the results obtained will be reported, with particular reference to the best configuration for each model.

## Imports

In [3]:
import itertools
import os.path
from typing import List, Dict

import pandas as pd

from src.models.config import param_layers, param_grid_mlp
from src.utils.const import MODEL_RESULTS_CSV, NETWORK_RESULT_CSV
from typing import Tuple

from src.utils.util_models import get_best_configuration_mlp
from src.visualization.visualize import barplot_multiple_columns
import numpy as np

### Useful path to data

In [4]:
RESULTS_FOLDER = os.path.join('..', MODEL_RESULTS_CSV)
MLP_RESULTS_FOLDER = os.path.join('..', NETWORK_RESULT_CSV)

## Read output csv

In [5]:
mlp_all = pd.read_csv(os.path.join(MLP_RESULTS_FOLDER, 'out_mlp_all.csv'))
mlp_batch_bad = pd.read_csv(os.path.join(MLP_RESULTS_FOLDER, 'out_bad_config_batch.csv'))
mlp_batch = pd.read_csv(os.path.join(MLP_RESULTS_FOLDER, 'out_mlp_batch.csv'))

svm_res = pd.read_csv(os.path.join(RESULTS_FOLDER, 'out_svm.csv'))
naive_res = pd.read_csv(os.path.join(RESULTS_FOLDER, 'best_out_naive_bayes.csv'))
tree_res = pd.read_csv(os.path.join(RESULTS_FOLDER, 'best_out_tree_based.csv'))

svm_val = pd.read_csv(os.path.join(RESULTS_FOLDER, 'out_grid_svm.csv'))
naive_val = pd.read_csv(os.path.join(RESULTS_FOLDER, 'out_grid_naive_bayes.csv'))
tree_val = pd.read_csv(os.path.join(RESULTS_FOLDER, 'out_grid_tree_based.csv'))

## Utils

In [ ]:
df_test_metric = pd.DataFrame()
df_train_metric=pd.DataFrame()

In [7]:
def mu_confidence_interval(data: np.ndarray) -> Dict:
    """
    Compute mean and t_student from a np.ndarray
    :param data: the DataFrame that contains movies_id and tmdb_id
    :return: Dict with metric values
    """
    t = 2.13
    mu = np.mean(data)
    standard_deviation = np.std(data)
    M = data.shape[0]
    t_student = t * standard_deviation / np.sqrt(M)
    first_interval = mu - t_student
    second_interval = mu + t_student
    return {
        'mu': mu,
        't_student': t_student,
        'first_interval': first_interval,
        'second_interval': second_interval
    }

In [ ]:
def summary_statistics_model(df_score: pd.DataFrame, dict_: Dict, model: str, train: bool = False) -> pd.DataFrame:
    if not train:
        print(
            f"Best configuration {model} mean metrics:\n"
            f"f1_score: {dict_['f1']['mu']} ±{dict_['f1']['t_student']}\n"
            f"loss: {dict_['loss']['mu']} ±{dict_['loss']['t_student']}\n"
            f"acc: {dict_['acc']['mu']} ±{dict_['acc']['t_student']}\n\n"
            f"Best hyperparams configuration:"
        )
        if model == "mlp" or model=='MLP':
            best_cfg_mlp_all = get_best_configuration_mlp(int(dict_['conf']), param_layers, param_grid_mlp)
            for idx, key in enumerate(param_layers.keys()):
                print(f"{key}: {best_cfg_mlp_all[idx]}")
            for idx, key in enumerate(param_grid_mlp.keys(), 7):
                print(f"{key}: {best_cfg_mlp_all[idx]}")
        else:
            print(f"{dict_['conf']}")

        new_test_score = pd.DataFrame({
            'model': [model],
            'f1_mu': [dict_['f1']['mu']],
            'acc_mu': [dict_['acc']['mu']],
            'loss_mu': [dict_['loss']['mu']],
            'f1_ci': [dict_['f1']['t_student']],
            'acc_ci': [dict_['acc']['t_student']],
            'loss_ci': [dict_['loss']['t_student']],
        })
        df_score = pd.concat([df_score, new_test_score], ignore_index=True)
    else:
        print(
            f"Best configuration {model} mean metrics:\n"
            f"train f1: {dict_['train_score']['mu']} ±{dict_['train_score']['t_student']}\n"
            f"validation f1: {dict_['val_score']['mu']} ±{dict_['val_score']['t_student']}\n"
        )
        new_test_score = pd.DataFrame({
            'model': [model],
            'train_score': [dict_['train_score']['mu']],
            'val_score': [dict_['val_score']['mu']],
            'train_ci': [dict_['train_score']['t_student']],
            'val_ci': [dict_['val_score']['t_student']]
        })
        df_score = pd.concat([df_score, new_test_score], ignore_index=True)
    return df_score

In [ ]:
def find_max_f1_cfg(df: pd.DataFrame) -> List:
    """
    Find best configuration based on f1-score
    :param data: the DataFrame that contains model outputs
    :return: List with best configuration indeces
    """
    cfg = []
    for fold in df['fold'].unique():
        idx = df[df['fold'] == fold]['f1_test'].idxmax()
        cfg.append(df.iloc[idx]['cfg'])
    cfgs = np.unique(np.array(cfg))
    return cfgs

In [ ]:
def find_best_conf(lst_conf, df: pd.DataFrame) -> Dict:
    conf = []
    for idx, cfg in enumerate(lst_conf):
        conf.append(
            {
                'f1': mu_confidence_interval(df[df['cfg'] == cfg]['f1_test']),
                'loss': mu_confidence_interval(df[df['cfg'] == cfg]['loss_test']),
                'acc': mu_confidence_interval(df[df['cfg'] == cfg]['acc_test'])
            }
        )
        conf[idx]['conf'] = cfg
        conf[idx]['acc']['mu'] /= 100
        conf[idx]['acc']['t_student'] /= 100
    max = conf[0]
    for elm in conf:
        if max['f1']['mu'] < elm['f1']['mu'] and max['f1']['t_student'] > elm['f1']['t_student']:
            max = elm
    return max

In [ ]:
def calculate_statistics_sklearn(df: pd.DataFrame, model: str) -> Dict:
    """
    Create a well-formatted Dict that contains the metrics
    :param df: the DataFrame that contains metrics
    :return: well-formatted Dict
    """
    res = {'f1': mu_confidence_interval(df[df['model'] == model]['f1_test']),
           'loss': mu_confidence_interval(df[df['model'] == model]['loss_test']),
           'acc': mu_confidence_interval(df[df['model'] == model]['acc_test']),
           'conf': df[df['model'] == model]['cfg'].unique()}
    return res

In [ ]:
def calculate_statistics_sklearn_train(df: pd.DataFrame, first:bool=True) -> Dict:
    if first:
        res = {'train_score': mu_confidence_interval(df['mean_train_score'].iloc[0:5]),
               'val_score': mu_confidence_interval(df['mean_test_score'].iloc[0:5]),
               }
    else:
        res = {'train_score': mu_confidence_interval(df['mean_train_score'].iloc[5:10]),
               'val_score': mu_confidence_interval(df['mean_test_score'].iloc[5:10]),
               }
    return res

In [ ]:
def calculate_statistics_mlp_train(cfg:int, df: pd.DataFrame) -> Dict:
    """
    Create a well-formatted Dict that contains the metrics
    :param cfg: the configuration index
    :param cfg: the DataFrame that contains metric
    :return: well-formatted Dict
    """
    res = {'train_score': mu_confidence_interval(df[df['cfg'] == cfg]['mean_f1_train']),
               'val_score': mu_confidence_interval(df[df['cfg'] == cfg]['mean_f1_val']),
               }
    return res

In [ ]:
def add_value_array(old_list: List, df_test: pd.DataFrame, col_name: str, last_idx: int) -> None:
    for model_name in df_test['model'].unique():
        if col_name == 'metrics':
            metric_value = df_test[df_test['model'] == model_name].iloc[:, 1:last_idx].iloc[0]
        else:
            metric_value = df_test[df_test['model'] == model_name].iloc[:, last_idx:].iloc[0]

        old_list.append(np.array(metric_value))

## Work with results

### Scikit-learn models

#### RandomForestClassifier

In [ ]:
res_random_forest_train = calculate_statistics_sklearn_train(tree_val)
df_train_metric = summary_statistics_model(df_train_metric, res_random_forest_train, 'Random forest classifier', train=True)

res_random_forest = calculate_statistics_sklearn(tree_res, 'random_forest_classifier')
df_test_metric = summary_statistics_model(df_test_metric, res_random_forest, 'Random forest classifier')

#### DecisionTreeClassifier

In [ ]:
res_decision_tree_train = calculate_statistics_sklearn_train(tree_val,first=False)
df_train_metric = summary_statistics_model(df_train_metric, res_decision_tree_train, 'Decision tree classifier', train=True)

res_decision_tree = calculate_statistics_sklearn(tree_res, 'decision_tree_classifier')
df_test_metric = summary_statistics_model(df_test_metric, res_decision_tree, 'Decision tree classifier')

#### GaussianNB

In [ ]:
res_gaussian_nb_train = calculate_statistics_sklearn_train(naive_val)
df_train_metric = summary_statistics_model(df_train_metric, res_gaussian_nb_train, 'GaussianNB', train=True)

res_gaussian_nb = calculate_statistics_sklearn(naive_res, 'gaussian_nb')
df_test_metric = summary_statistics_model(df_test_metric, res_gaussian_nb, 'GaussianNB')

#### QDA

In [ ]:
res_qda_train = calculate_statistics_sklearn_train(naive_val,first=False)
df_train_metric = summary_statistics_model(df_train_metric, res_qda_train, 'QDA', train=True)

res_qda = calculate_statistics_sklearn(naive_res, 'qda')
df_test_metric = summary_statistics_model(df_test_metric, res_qda, 'QDA')

#### SVM

In [ ]:
res_svm_train = calculate_statistics_sklearn_train(svm_val)
df_train_metric = summary_statistics_model(df_train_metric, res_svm_train, 'SVM', train=True)

res_svm = calculate_statistics_sklearn(svm_res, 'svc')
df_test_metric = summary_statistics_model(df_test_metric, res_svm, 'SVM')

### MovieNet (MLP)

In [ ]:
best_cfg = find_max_f1_cfg(mlp_all)
print(f'Indices of the best configurations: {best_cfg}')

In [ ]:
res_mlp_train = calculate_statistics_mlp_train(3,mlp_all)
df_train_metric = summary_statistics_model(df_train_metric, res_mlp_train, 'MLP', train=True)

res_mlp = find_best_conf(best_cfg, mlp_all)
df_test_metric = summary_statistics_model(df_test_metric, res_mlp, "MLP")

### Plot results

#### Train & Validation

In [ ]:
metrics = []
y_errs = []
add_value_array(metrics, df_train_metric, 'metrics', 3)
add_value_array(y_errs, df_train_metric, 'interval', 3)

In [ ]:
barplot_multiple_columns(groups=['Training', 'Validation'],
                         elements_group=df_train_metric['model'].unique(), data=metrics, yerr=y_errs,
                         title='Train/val metrics',
                         upper_title=False)

#### Test

In [ ]:
metrics = []
y_errs = []
add_value_array(metrics, df_test_metric, 'metrics', 4)
add_value_array(y_errs, df_test_metric, 'interval', 4)

In [ ]:
barplot_multiple_columns(groups=['F1 score ', 'Accuracy', 'Loss (lower is better)'],
                         elements_group=df_test_metric['model'].unique(), data=metrics, yerr=y_errs,
                         title='Test metrics',
                         upper_title=False)

### working with batch

In [9]:
from src.models.config import param_layers_batch, param_grid_mlp_batch

df = pd.DataFrame()
for idx in mlp_batch['cfg'].unique():
    config = get_best_configuration_mlp(idx, param_layers_batch, param_grid_mlp_batch)
    batch_size = config[9]
    f1_dict = mu_confidence_interval(mlp_batch[mlp_batch['cfg'] == idx]['f1_test'])
    loss_dict = mu_confidence_interval(mlp_batch[mlp_batch['cfg'] == idx]['loss_test'])
    acc_dict = mu_confidence_interval(mlp_batch[mlp_batch['cfg'] == idx]['acc_test'])

    new_sample = pd.DataFrame({
        'index': [idx],
        'batch_size': [batch_size],
        'f1_mean': [f1_dict['mu']],
        'f1_confidence': [f1_dict['t_student']],
        'loss_mean': [loss_dict['mu']],
        'loss_confidence': [loss_dict['t_student']],
        'acc_mean': [acc_dict['mu']],
        'acc_confidence': [acc_dict['t_student']]

    })
    df = pd.concat([df, new_sample], ignore_index=True)
print(df)

    index  batch_size   f1_mean  f1_confidence  loss_mean  loss_confidence  \
0       0           8  0.285382       0.242427   8.021037         5.736820   
1       1          16  0.310350       0.229459   6.453717         4.000400   
2       2          32  0.400204       0.270151   5.942804         6.218700   
3       3          64  0.446364       0.233992   2.074904         1.458697   
4       4         128  0.326819       0.159141   1.850459         0.682113   
5       5         256  0.121477       0.044247   2.276308         0.016917   
6       6         512  0.170163       0.012856   6.918986         2.838452   
7       7        1024  0.175928       0.055172   6.227965         1.850419   
8       8        2048  0.303729       0.221356   5.000649         2.725434   
9       9        4096  0.225762       0.032366   3.205096         0.673349   
10     10        8192  0.364835       0.136903   1.696874         0.454730   
11     11       16384  0.142564       0.018369   2.292822       